In [1]:
import sqlite3
import tkinter as tk
from tkinter import messagebox
import logging
import sys

# ---------------- LOGGING CONFIG ----------------
logging.basicConfig(
    filename="soulsense.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("Application started")

# ---------------- DATABASE SETUP ----------------
try:
    conn = sqlite3.connect("soulsense_db")
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS scores (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT,
        age INTEGER,
        total_score INTEGER
    )
    """)
    conn.commit()

    logging.info("Database initialized successfully")

except Exception:
    logging.error("Database initialization failed", exc_info=True)
    sys.exit(1)

# ---------------- LOAD QUESTIONS ----------------
try:
    with open("question_bank.txt", "r") as question_bank:
        questions = [line.strip() for line in question_bank]

except Exception:
    logging.error("Failed to load questions from question_bank.txt", exc_info=True)
    sys.exit(1)

questions = questions[:10]
    
# ---------------- GUI APPLICATION ----------------
class SoulSenseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Soul Sense EQ Test")

        self.username = ""
        self.age = None
        self.current_question = 0
        self.responses = []

        self.create_username_screen()

    def create_username_screen(self):
        self.clear_screen()

        tk.Label(self.root, text="Enter Your Name:", font=("Arial", 14)).pack(pady=10)
        self.name_entry = tk.Entry(self.root, font=("Arial", 14))
        self.name_entry.pack()

        tk.Label(self.root, text="Enter Your Age (optional):", font=("Arial", 14)).pack(pady=10)
        self.age_entry = tk.Entry(self.root, font=("Arial", 14))
        self.age_entry.pack()

        tk.Button(self.root, text="Start Test", command=self.start_test).pack(pady=20)

    def validate_name_input(self, name):
        if name == "":
            return False, "Please enter your name."
        if not all(c.isalpha() or c.isspace() for c in name):
            return False, "Name should contain only letters and spaces."
        return True, None

    def validate_age_input(self, age_str):
        if age_str == "":
            return True, None, None
        try:
            age = int(age_str)
            if age < 1 or age > 120:
                return False, None, "Age must be between 1 and 120."
            return True, age, None
        except ValueError:
            return False, None, "Age must be a number."

    def start_test(self):
        self.username = self.name_entry.get().strip()
        age_str = self.age_entry.get().strip()

        valid, msg = self.validate_name_input(self.username)
        if not valid:
            messagebox.showwarning("Input Error", msg)
            logging.warning(f"Invalid username input: {self.username}")
            return

        valid, age, msg = self.validate_age_input(age_str)
        if not valid:
            messagebox.showwarning("Input Error", msg)
            logging.warning(f"Invalid age input: {age_str}")
            return

        self.age = age
        logging.info(f"User session started | username={self.username} | age={self.age}")
        self.show_question()

    def show_question(self):
        self.clear_screen()

        if self.current_question < len(questions):
            q_text = questions[self.current_question]

            tk.Label(
                self.root,
                text=f"Q{self.current_question + 1}: {q_text}",
                wraplength=400,
                font=("Arial", 14)
            ).pack(pady=20)

            self.answer_var = tk.IntVar()

            for val, text in enumerate(
                ["Never (1)", "Sometimes (2)", "Often (3)", "Always (4)"], start=1
            ):
                tk.Radiobutton(
                    self.root, text=text, variable=self.answer_var, value=val
                ).pack(anchor="w", padx=50)

            tk.Button(self.root, text="Next", command=self.save_answer).pack(pady=20)
        else:
            self.finish_test()

    def save_answer(self):
        ans = self.answer_var.get()
        if ans == 0:
            messagebox.showwarning("Input Error", "Please select an answer.")
            return

        self.responses.append(ans)
        self.current_question += 1
        self.show_question()

    def finish_test(self):
        total_score = sum(self.responses)

        try:
            cursor.execute(
                "INSERT INTO scores (username, age, total_score) VALUES (?, ?, ?)",
                (self.username, self.age, total_score)
            )
            conn.commit()
            logging.info(
                f"Score saved successfully | user={self.username} | score={total_score}"
            )

        except Exception:
            logging.error("Failed to save final score", exc_info=True)
            messagebox.showerror("Database Error", "Failed to save your score.")
            return

        self.clear_screen()

        tk.Label(
            self.root,
            text=f"Your total EQ score is: {total_score} / 80",
            font=("Arial", 14)
        ).pack(pady=20)

        tk.Button(self.root, text="Exit", command=self.exit_test).pack(pady=20)

    def exit_test(self):
        try:
            conn.close()
            logging.info("Database connection closed")
        except Exception:
            logging.error("Error closing database", exc_info=True)

        logging.info("Application exited gracefully")
        self.root.quit()

    def clear_screen(self):
        for widget in self.root.winfo_children():
            widget.destroy()

# ---------------- MAIN LOOP ----------------
if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("500x300")

    app = SoulSenseApp(root)
    root.protocol("WM_DELETE_WINDOW", app.exit_test)

    root.mainloop()


In [1]:
import sqlite3
import tkinter as tk
from tkinter import messagebox
import logging
import sys
import os

# ---------------- LOGGING SETUP ----------------
logging.basicConfig(
    filename="soulsense.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("Application started")

# ---------------- DATABASE INIT ----------------
def ensure_scores_schema(cursor):
    cursor.execute("PRAGMA table_info(scores)")
    columns = [col[1] for col in cursor.fetchall()]

    if "age" not in columns:
        logging.info("Migrating scores table: adding age column")
        cursor.execute("ALTER TABLE scores ADD COLUMN age INTEGER")

try:
    conn = sqlite3.connect("soulsense_db")
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS scores (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT,
            total_score INTEGER
        )
    """)

    ensure_scores_schema(cursor)
    conn.commit()

    logging.info("Database initialized successfully")

except Exception:
    logging.critical("Fatal error during database initialization", exc_info=True)
    messagebox.showerror("Fatal Error", "Unable to initialize database.")
    sys.exit(1)

# ---------------- LOAD QUESTIONS ----------------
try:
    BASE_DIR = os.getcwd()
    QUESTION_FILE = os.path.join(BASE_DIR, "question_bank.txt")

    logging.info("Loading questions from %s", QUESTION_FILE)

    with open(QUESTION_FILE, "r") as f:
        questions = [line.strip() for line in f if line.strip()]
        
    '''
    to test only selecting 10 questions
    questions = questions[:10]
    '''    
    
    if not questions:
        raise ValueError("Question bank is empty")

    logging.info("Questions loaded successfully | count=%s", len(questions))

except Exception:
    logging.critical("Failed to load questions", exc_info=True)
    messagebox.showerror(
        "Fatal Error",
        "Question bank could not be loaded.\nApplication will close."
    )
    sys.exit(1)

# ---------------- GUI APPLICATION ----------------
class SoulSenseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Soul Sense EQ Test")

        self.username = ""
        self.age = None
        self.current_question = 0
        self.responses = []

        self.create_username_screen()

    # ---------- SCREENS ----------
    def create_username_screen(self):
        self.clear_screen()

        tk.Label(self.root, text="Enter Your Name:", font=("Arial", 14)).pack(pady=10)
        self.name_entry = tk.Entry(self.root, font=("Arial", 14))
        self.name_entry.pack(pady=5)

        tk.Label(self.root, text="Enter Your Age (optional):", font=("Arial", 14)).pack(pady=5)
        self.age_entry = tk.Entry(self.root, font=("Arial", 14))
        self.age_entry.pack(pady=5)

        tk.Button(self.root, text="Start Test", command=self.start_test).pack(pady=15)

    # ---------- VALIDATION ----------
    def validate_name_input(self, name):
        if not name:
            return False, "Please enter your name."
        if not all(c.isalpha() or c.isspace() for c in name):
            return False, "Name must contain only letters and spaces."
        return True, None

    def validate_age_input(self, age_str):
        if age_str == "":
            return True, None, None
        try:
            age = int(age_str)
            if not (1 <= age <= 120):
                return False, None, "Age must be between 1 and 120."
            return True, age, None
        except ValueError:
            return False, None, "Age must be numeric."

    # ---------- FLOW ----------
    def start_test(self):
        try:
            self.username = self.name_entry.get().strip()
            age_str = self.age_entry.get().strip()

            ok, err = self.validate_name_input(self.username)
            if not ok:
                logging.warning("Invalid name input: %s", self.username)
                messagebox.showwarning("Input Error", err)
                return

            ok, age, err = self.validate_age_input(age_str)
            if not ok:
                logging.warning("Invalid age input: %s", age_str)
                messagebox.showwarning("Input Error", err)
                return

            self.age = age
            logging.info("User session started | username=%s | age=%s", self.username, self.age)

            self.show_question()

        except Exception:
            self.handle_fatal_error("Error starting test")

    def show_question(self):
        try:
            self.clear_screen()

            if self.current_question >= len(questions):
                self.finish_test()
                return

            q = questions[self.current_question]
            tk.Label(self.root, text=f"Q{self.current_question+1}: {q}", wraplength=400).pack(pady=20)

            self.answer_var = tk.IntVar()
            for val, txt in enumerate(["Never", "Sometimes", "Often", "Always"], 1):
                tk.Radiobutton(self.root, text=f"{txt} ({val})", variable=self.answer_var, value=val).pack(anchor="w", padx=50)

            tk.Button(self.root, text="Next", command=self.save_answer).pack(pady=15)

        except Exception:
            self.handle_fatal_error("Error displaying question")

    def save_answer(self):
        try:
            ans = self.answer_var.get()
            if ans == 0:
                logging.warning("No answer selected")
                messagebox.showwarning("Input Error", "Please select an answer.")
                return

            self.responses.append(ans)
            self.current_question += 1
            self.show_question()

        except Exception:
            self.handle_fatal_error("Error saving answer")

    def finish_test(self):
        try:
            total_score = sum(self.responses)

            cursor.execute(
                "INSERT INTO scores (username, age, total_score) VALUES (?, ?, ?)",
                (self.username, self.age, total_score)
            )
            conn.commit()

            logging.info("Score saved | user=%s | score=%s", self.username, total_score)

            self.clear_screen()
            tk.Label(self.root, text=f"Score: {total_score}/80", font=("Arial", 16)).pack(pady=20)
            tk.Button(self.root, text="Exit", command=self.force_exit).pack(pady=10)

        except Exception:
            self.handle_fatal_error("Failed to finish test")

    # ---------- EXIT ----------
    def force_exit(self):
        try:
            conn.close()
            logging.info("Database connection closed")
        except Exception:
            logging.error("Error closing database", exc_info=True)
        finally:
            logging.info("Application exited")
            self.root.destroy()
            sys.exit(0)

    def handle_fatal_error(self, msg):
        logging.critical(msg, exc_info=True)
        messagebox.showerror("Fatal Error", "A critical error occurred. App will close.")
        self.force_exit()

    def clear_screen(self):
        for w in self.root.winfo_children():
            w.destroy()

# ---------------- MAIN ----------------
try:
    root = tk.Tk()
    root.geometry("500x300")
    app = SoulSenseApp(root)
    root.protocol("WM_DELETE_WINDOW", app.force_exit)
    root.mainloop()

except Exception:
    logging.critical("Unhandled crash", exc_info=True)
    try:
        conn.close()
    except Exception:
        pass
    sys.exit(1)


SystemExit: 0

C:\Users\akhil\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
